# Old GDP Real-Time Dataset

**Author:** Jason Cruz  
**Last updated:** 08/13/2025  
**Python version:** 3.12  
**Project:** Rationality and Nowcasting on Peruvian GDP Revisions  

---
## 📌 Summary
This notebook documents the step-by-step **construction of real-time datasets** for **Peruvian GDP revisions** since 2013–PRESENT. It covers:

1. **Downloading PDFs** (actually Weekly Reports (WR)) from the Central Reserve Bank of Peru's website.
2. **Generating PDF inputs** by shorten them in order to keep key pages containing required tables where GDP growth rates are in.
3. **Cleaning-up data** extracted from input PDFs.
4. **Concatenating real-time datasets across years by frequency** 
5. **Storing RTD to SQL** for availability to users upon request and further analysis.

🌐 **Main Data Source:** [BCRP Weekly Report](https://www.bcrp.gob.pe/publicaciones/nota-semanal.html) (📰 WR, from here on)  
Any questions or issues regarding the coding, please email [Jason 📨](mailto:jj.cruza@up.edu.pe)  

---

## 🛠️ Libraries

If you don't have the libraries below, please use the following code (as example) to install the required libraries.

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

Check out Python information

In [ ]:
import sys
import platform

print("🐍 Python Information")
print(f"  Version  : {sys.version.split()[0]}")
print(f"  Compiler : {platform.python_compiler()}")
print(f"  Build    : {platform.python_build()}")
print(f"  OS       : {platform.system()} {platform.release()}")

**Import helper functions**

> ⚠️ Please, check the script `new_gdp_datasets_functions.py` which contains all the functions required by this _jupyter notebook_. The functions there are ordered according to the sections of this jupyter notebok.

In [ ]:
from gdp_rtd_pipeline import *

## ⚙️ Initial set-up

Before preprocessing new GDP releases data, we will:

* **Create necessary folders** for storing inputs, outputs, logs, and screenshots.
* **Connect to the PostgreSQL database** containing GDP revisions datasets.
* **Import helper functions** from `new_gdp_datasets_functions.py`.

**Create necessary folders**

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()
user_input = input("Enter relative path (default='.'): ").strip() or "."
target_path = (PROJECT_ROOT / user_input).resolve()
target_path.mkdir(parents=True, exist_ok=True)
print(f"📂 Using path: {target_path}")

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Data cleaning</span></h1>

In [ ]:
# Define base folder for saving vintages data (.csv)
data_folder = 'data'

# Define subfolder for saving 
input_data_subfolder = os.path.join(data_folder, 'input')

# Define subfolder for saving 
output_data_subfolder = os.path.join(data_folder, 'output')

# Create all required folders (if they do not already exist) and confirm creation
for folder in [data_folder, input_data_subfolder, output_data_subfolder]:
    os.makedirs(folder, exist_ok=True)
    print(f"📂 {folder} created")

In [ ]:
raw_1, clean_1, vintages_1 = table_1_cleaner(
    input_pdf_folder = input_pdf_subfolder,
    record_folder = record_folder,
    record_txt = '3_created_vintages_tab_1.txt',
    persist = True,
    persist_folder = input_data_subfolder,
    pipeline_version = "s3.0.0",
)

In [ ]:
raw_1.keys()

In [ ]:
clean_1.keys()

In [ ]:
vintages_1.keys()

In [ ]:
raw_1['ns_04_2022_1']

In [ ]:
clean_1['ns_04_2022_1']

In [ ]:
vintages_1['ns_04_2022_1']

# Checking the cleaning version out

In [ ]:
df100 = vintages_1["ns_04_2022_1"]
print(df100.attrs)
# {'pipeline_version': 's3.0.0'}


In [ ]:
vintages_1["ns_04_2022_1"].attrs

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Extracting tables and data cleanup
    </span>
    </h2>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
PENDING INTRO TEXT
</p>
<div/>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 1.</span> Extraction and cleaning of data from tables on monthly real GDP growth rates.
    </span>
    </h3>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <b>"ns_dates"</b> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

In [11]:
# Set the locale to Spanish
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Dictionary to store generated DataFrames
old_dataframes_dict_1 = {}

# Path for the processed folders log file
record_path = 'dataframes_record/old_processed_folders_1.txt'

# Function to correct month names
def correct_month_name(month):
    months_mapping = {
        'setiembre': 'septiembre',
        # Add more mappings as needed for other month names
    }
    return months_mapping.get(month, month)

# Function to register processed folder
def register_processed_folder(folder, num_processed_files):
    with open(record_path, 'a') as file:
        file.write(f"{folder}:{num_processed_files}\n")

# Function to check if folder has been processed
def folder_processed(folder):
    if not os.path.exists(record_path):
        return False
    with open(record_path, 'r') as file:
        for line in file:
            if line.startswith(folder):
                return True
    return False

# Function to fetch date from database
def get_date(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    date_result = pd.read_sql(query, engine)
    return date_result.iloc[0, 0] if not date_result.empty else None

def process_csv(csv_path, engine):
    old_tables_dict_1 = {}  # Local dictionary for each CSV
    table_counter = 1

    filename = os.path.basename(csv_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No matches found for id_ns and year in filename:", filename)
        return None, None, None, None # Return None for tables_dict_1 as well

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    dataframe_name = f"{new_filename}_{table_counter}"
    old_tables_dict_1[dataframe_name] = df.copy()

    # Apply cleanup functions to a copy of the DataFrame
    df_clean = df.copy()

    # Conditional cleaning based on the first column
    if df_clean.columns[1] == 'economic_sectors':
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)
    else:
        # Cleaning functions (set as required)
        df_clean = clean_column_names(df_clean)
        df_clean = adjust_column_names(df_clean)
        df_clean = drop_rare_caracter_row(df_clean)
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        df_clean = reset_index(df_clean)
        df_clean = remove_digit_slash(df_clean)
        df_clean = replace_var_perc_first_column(df_clean)
        df_clean = replace_var_perc_last_columns(df_clean)
        df_clean = replace_number_moving_average(df_clean)
        df_clean = relocate_last_column(df_clean)
        df_clean = clean_first_row(df_clean)
        df_clean = find_year_column(df_clean)
        year_columns = extract_years(df_clean)
        df_clean = get_months_sublist_list(df_clean, year_columns)
        df_clean = first_row_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)

    # Add the column 'year' to the clean DataFrame
    df_clean.insert(0, 'year', year)
    
    # Add the column 'id_ns' to the clean DataFrame
    df_clean.insert(1, 'id_ns', id_ns)
    
    # Get corresponding date from database
    date = get_date(df_clean, engine)
    if date:
        # Add 'date' column to cleaned DataFrame
        df_clean.insert(2, 'date', date)
    else:
        print("Date not found in database for id_ns:", id_ns, "and year:", year)

    # Store cleaned DataFrame in old_dataframes_dict_1
    old_dataframes_dict_1[dataframe_name] = df_clean

    return id_ns, year, old_tables_dict_1

# Function to process folder
def process_folder(folder, engine):
    print(f"Processing folder {os.path.basename(folder)}")
    csv_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]

    num_csv_processed = 0
    num_dataframes_generated = 0

    table_counter = 1  # Initialize table counter here
    old_tables_dict_1 = {}  # Declare old_tables_dict_1 outside main loop
    
    for csv_file in csv_files:
        id_ns, year, tables_dict_temp = process_csv(csv_file, engine)

        if tables_dict_temp:
            for dataframe_name, df in tables_dict_temp.items():
                file_name = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                dataframe_name = f"{file_name}_1"
                
                # Store raw DataFrame in old_tables_dict_1
                old_tables_dict_1[dataframe_name] = df.copy()
                
                # Apply cleanup functions to a copy of the DataFrame
                df_clean = df.copy()

                # Conditional cleaning based on the first column
                if df_clean.columns[1] == 'economic_sectors':
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                else:
                    # Cleaning functions (set as required)
                    df_clean = clean_column_names(df_clean)
                    df_clean = adjust_column_names(df_clean)
                    df_clean = drop_rare_caracter_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = replace_var_perc_first_column(df_clean)
                    df_clean = replace_var_perc_last_columns(df_clean)
                    df_clean = replace_number_moving_average(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = find_year_column(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = get_months_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)

                # Add the column 'year' to the clean DataFrame
                df_clean.insert(0, 'year', year)

                # Add the column 'id_ns' to the clean DataFrame
                df_clean.insert(1, 'id_ns', id_ns)

                # Get corresponding date from database
                date = get_date(df_clean, engine)
                if date:
                    # Add 'date' column to cleaned DataFrame
                    df_clean.insert(2, 'date', date)
                else:
                    print("Date not found in database for id_ns:", id_ns, "and year:", year)

                # Store cleaned DataFrame in old_dataframes_dict_1
                old_dataframes_dict_1[dataframe_name] = df_clean

                print(f'  {table_counter}. The dataframe generated for the {csv_file} file is: {dataframe_name}')
                num_dataframes_generated += 1
                table_counter += 1  # Increment table counter here
        
        num_csv_processed += 1  # Increment number of processed CSV for each CSV in folder

    return num_csv_processed, num_dataframes_generated, old_tables_dict_1

def process_folders():
    csv_folder = table_1_folder
    folders = [os.path.join(csv_folder, d) for d in os.listdir(csv_folder) if os.path.isdir(os.path.join(csv_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_1 = {}  # Initialize old_tables_dict_1 here
    
    for folder in folders:
        if folder_processed(folder):
            print(f"Folder {folder} has already been processed.")
            continue
        
        num_csv_processed, num_dataframes_generated, tables_dict_temp = process_folder(folder, engine)
        
        # Update old_tables_dict_1 with values returned from process_folder()
        old_tables_dict_1.update(tables_dict_temp)
        
        register_processed_folder(folder, num_csv_processed)

        # Ask user if they want to continue with next folder
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Ensure the messagebox is in front
        message = f"Process {folder} complete. Processed {num_csv_processed} CSV(s) and generated {num_dataframes_generated} DataFrame(s). Continue with next folder?"
        if not messagebox.askyesno("Continue?", message):
            break
            
    print("Processing completed for all folders.")  # Add a message to indicate completion
    
    return old_tables_dict_1

if __name__ == "__main__":
    engine = create_sqlalchemy_engine()
    old_tables_dict_1 = process_folders()


Processing folder 1994
  1. The dataframe generated for the input_data\table_1\1994\ns-01-1994.csv file is: ns_01_1994_1
  2. The dataframe generated for the input_data\table_1\1994\ns-02-1994.csv file is: ns_02_1994_1
  3. The dataframe generated for the input_data\table_1\1994\ns-03-1994.csv file is: ns_03_1994_1
  4. The dataframe generated for the input_data\table_1\1994\ns-04-1994.csv file is: ns_04_1994_1
  5. The dataframe generated for the input_data\table_1\1994\ns-05-1994.csv file is: ns_05_1994_1
  6. The dataframe generated for the input_data\table_1\1994\ns-06-1994.csv file is: ns_06_1994_1
  7. The dataframe generated for the input_data\table_1\1994\ns-07-1994.csv file is: ns_07_1994_1
  8. The dataframe generated for the input_data\table_1\1994\ns-08-1994.csv file is: ns_08_1994_1
  9. The dataframe generated for the input_data\table_1\1994\ns-09-1994.csv file is: ns_09_1994_1
  10. The dataframe generated for the input_data\table_1\1994\ns-10-1994.csv file is: ns_10_199

  38. The dataframe generated for the input_data\table_1\1995\ns-38-1995.csv file is: ns_38_1995_1
  39. The dataframe generated for the input_data\table_1\1995\ns-39-1995.csv file is: ns_39_1995_1
  40. The dataframe generated for the input_data\table_1\1995\ns-40-1995.csv file is: ns_40_1995_1
  41. The dataframe generated for the input_data\table_1\1995\ns-41-1995.csv file is: ns_41_1995_1
  42. The dataframe generated for the input_data\table_1\1995\ns-42-1995.csv file is: ns_42_1995_1
  43. The dataframe generated for the input_data\table_1\1995\ns-43-1995.csv file is: ns_43_1995_1
  44. The dataframe generated for the input_data\table_1\1995\ns-44-1995.csv file is: ns_44_1995_1
  45. The dataframe generated for the input_data\table_1\1995\ns-45-1995.csv file is: ns_45_1995_1
  46. The dataframe generated for the input_data\table_1\1995\ns-46-1995.csv file is: ns_46_1995_1
  47. The dataframe generated for the input_data\table_1\1995\ns-47-1995.csv file is: ns_47_1995_1
  48. The 

  18. The dataframe generated for the input_data\table_1\1997\ns-18-1997.csv file is: ns_18_1997_1
  19. The dataframe generated for the input_data\table_1\1997\ns-19-1997.csv file is: ns_19_1997_1
  20. The dataframe generated for the input_data\table_1\1997\ns-20-1997.csv file is: ns_20_1997_1
  21. The dataframe generated for the input_data\table_1\1997\ns-21-1997.csv file is: ns_21_1997_1
  22. The dataframe generated for the input_data\table_1\1997\ns-22-1997.csv file is: ns_22_1997_1
  23. The dataframe generated for the input_data\table_1\1997\ns-23-1997.csv file is: ns_23_1997_1
  24. The dataframe generated for the input_data\table_1\1997\ns-24-1997.csv file is: ns_24_1997_1
  25. The dataframe generated for the input_data\table_1\1997\ns-25-1997.csv file is: ns_25_1997_1
  26. The dataframe generated for the input_data\table_1\1997\ns-26-1997.csv file is: ns_26_1997_1
  27. The dataframe generated for the input_data\table_1\1997\ns-27-1997.csv file is: ns_27_1997_1
  28. The 

  5. The dataframe generated for the input_data\table_1\1999\ns-05-1999.csv file is: ns_05_1999_1
  6. The dataframe generated for the input_data\table_1\1999\ns-06-1999.csv file is: ns_06_1999_1
  7. The dataframe generated for the input_data\table_1\1999\ns-07-1999.csv file is: ns_07_1999_1
  8. The dataframe generated for the input_data\table_1\1999\ns-08-1999.csv file is: ns_08_1999_1
  9. The dataframe generated for the input_data\table_1\1999\ns-09-1999.csv file is: ns_09_1999_1
  10. The dataframe generated for the input_data\table_1\1999\ns-10-1999.csv file is: ns_10_1999_1
  11. The dataframe generated for the input_data\table_1\1999\ns-11-1999.csv file is: ns_11_1999_1
  12. The dataframe generated for the input_data\table_1\1999\ns-12-1999.csv file is: ns_12_1999_1
  13. The dataframe generated for the input_data\table_1\1999\ns-13-1999.csv file is: ns_13_1999_1
  14. The dataframe generated for the input_data\table_1\1999\ns-14-1999.csv file is: ns_14_1999_1
  15. The dataf

  42. The dataframe generated for the input_data\table_1\2000\ns-42-2000.csv file is: ns_42_2000_1
  43. The dataframe generated for the input_data\table_1\2000\ns-43-2000.csv file is: ns_43_2000_1
  44. The dataframe generated for the input_data\table_1\2000\ns-44-2000.csv file is: ns_44_2000_1
  45. The dataframe generated for the input_data\table_1\2000\ns-45-2000.csv file is: ns_45_2000_1
  46. The dataframe generated for the input_data\table_1\2000\ns-46-2000.csv file is: ns_46_2000_1
  47. The dataframe generated for the input_data\table_1\2000\ns-47-2000.csv file is: ns_47_2000_1
  48. The dataframe generated for the input_data\table_1\2000\ns-48-2000.csv file is: ns_48_2000_1
  49. The dataframe generated for the input_data\table_1\2000\ns-49-2000.csv file is: ns_49_2000_1
Processing folder 2001
  1. The dataframe generated for the input_data\table_1\2001\ns-01-2001.csv file is: ns_01_2001_1
  2. The dataframe generated for the input_data\table_1\2001\ns-02-2001.csv file is: ns

...

  17. The dataframe generated for the input_data\table_1\2004\ns-17-2004.csv file is: ns_17_2004_1
  18. The dataframe generated for the input_data\table_1\2004\ns-18-2004.csv file is: ns_18_2004_1
  19. The dataframe generated for the input_data\table_1\2004\ns-19-2004.csv file is: ns_19_2004_1
  20. The dataframe generated for the input_data\table_1\2004\ns-20-2004.csv file is: ns_20_2004_1
  21. The dataframe generated for the input_data\table_1\2004\ns-21-2004.csv file is: ns_21_2004_1
  22. The dataframe generated for the input_data\table_1\2004\ns-22-2004.csv file is: ns_22_2004_1
  23. The dataframe generated for the input_data\table_1\2004\ns-23-2004.csv file is: ns_23_2004_1
  24. The dataframe generated for the input_data\table_1\2004\ns-24-2004.csv file is: ns_24_2004_1
  25. The dataframe generated for the input_data\table_1\2004\ns-25-2004.csv file is: ns_25_2004_1
  26. The dataframe generated for the input_data\table_1\2004\ns-26-2004.csv file is: ns_26_2004_1
  27. The 

  6. The dataframe generated for the input_data\table_1\2006\ns-06-2006.csv file is: ns_06_2006_1
  7. The dataframe generated for the input_data\table_1\2006\ns-07-2006.csv file is: ns_07_2006_1
  8. The dataframe generated for the input_data\table_1\2006\ns-08-2006.csv file is: ns_08_2006_1
  9. The dataframe generated for the input_data\table_1\2006\ns-09-2006.csv file is: ns_09_2006_1
  10. The dataframe generated for the input_data\table_1\2006\ns-10-2006.csv file is: ns_10_2006_1
  11. The dataframe generated for the input_data\table_1\2006\ns-11-2006.csv file is: ns_11_2006_1
  12. The dataframe generated for the input_data\table_1\2006\ns-12-2006.csv file is: ns_12_2006_1
  13. The dataframe generated for the input_data\table_1\2006\ns-13-2006.csv file is: ns_13_2006_1
  14. The dataframe generated for the input_data\table_1\2006\ns-14-2006.csv file is: ns_14_2006_1
  15. The dataframe generated for the input_data\table_1\2006\ns-15-2006.csv file is: ns_15_2006_1
  16. The data

  45. The dataframe generated for the input_data\table_1\2007\ns-45-2007.csv file is: ns_45_2007_1
  46. The dataframe generated for the input_data\table_1\2007\ns-46-2007.csv file is: ns_46_2007_1
  47. The dataframe generated for the input_data\table_1\2007\ns-47-2007.csv file is: ns_47_2007_1
  48. The dataframe generated for the input_data\table_1\2007\ns-48-2007.csv file is: ns_48_2007_1
  49. The dataframe generated for the input_data\table_1\2007\ns-49-2007.csv file is: ns_49_2007_1
Processing folder 2008
  1. The dataframe generated for the input_data\table_1\2008\ns-01-2008.csv file is: ns_01_2008_1
  2. The dataframe generated for the input_data\table_1\2008\ns-02-2008.csv file is: ns_02_2008_1
  3. The dataframe generated for the input_data\table_1\2008\ns-03-2008.csv file is: ns_03_2008_1
  4. The dataframe generated for the input_data\table_1\2008\ns-04-2008.csv file is: ns_04_2008_1
  5. The dataframe generated for the input_data\table_1\2008\ns-05-2008.csv file is: ns_05

  32. The dataframe generated for the input_data\table_1\2009\ns-32-2009.csv file is: ns_32_2009_1
  33. The dataframe generated for the input_data\table_1\2009\ns-33-2009.csv file is: ns_33_2009_1
  34. The dataframe generated for the input_data\table_1\2009\ns-34-2009.csv file is: ns_34_2009_1
  35. The dataframe generated for the input_data\table_1\2009\ns-35-2009.csv file is: ns_35_2009_1
  36. The dataframe generated for the input_data\table_1\2009\ns-36-2009.csv file is: ns_36_2009_1
  37. The dataframe generated for the input_data\table_1\2009\ns-37-2009.csv file is: ns_37_2009_1
  38. The dataframe generated for the input_data\table_1\2009\ns-38-2009.csv file is: ns_38_2009_1
  39. The dataframe generated for the input_data\table_1\2009\ns-39-2009.csv file is: ns_39_2009_1
  40. The dataframe generated for the input_data\table_1\2009\ns-40-2009.csv file is: ns_40_2009_1
  41. The dataframe generated for the input_data\table_1\2009\ns-41-2009.csv file is: ns_41_2009_1
  42. The 

  20. The dataframe generated for the input_data\table_1\2011\ns-20-2011.csv file is: ns_20_2011_1
  21. The dataframe generated for the input_data\table_1\2011\ns-21-2011.csv file is: ns_21_2011_1
  22. The dataframe generated for the input_data\table_1\2011\ns-22-2011.csv file is: ns_22_2011_1
  23. The dataframe generated for the input_data\table_1\2011\ns-23-2011.csv file is: ns_23_2011_1
  24. The dataframe generated for the input_data\table_1\2011\ns-24-2011.csv file is: ns_24_2011_1
  25. The dataframe generated for the input_data\table_1\2011\ns-25-2011.csv file is: ns_25_2011_1
  26. The dataframe generated for the input_data\table_1\2011\ns-26-2011.csv file is: ns_26_2011_1
  27. The dataframe generated for the input_data\table_1\2011\ns-27-2011.csv file is: ns_27_2011_1
  28. The dataframe generated for the input_data\table_1\2011\ns-28-2011.csv file is: ns_28_2011_1
  29. The dataframe generated for the input_data\table_1\2011\ns-29-2011.csv file is: ns_29_2011_1
  30. The 

In [12]:
old_tables_dict_1['ns_13_1997_1'].head(30)

,sectores_economicos,economic_sectors,1995_nov,1995_dic,1995_year,1996_ene,1996_feb,1996_mar,1996_abr,1996_may,1996_jun,1996_jul,1996_ago,1996_sep,1996_oct,1996_nov,1996_dic,1996_ene_dic,1997_ene
0,agropecuario,agriculture and livestock,-0.1,3.5,7.6,-3.6,0.3,2.3,7.2,2.8,14.4,3.8,9.8,3.3,4.7,5.4,9.3,5.6,17.1
1,agricola,agriculture,-7.8,1.7,6.1,-7.4,1.6,2.3,8.4,2.5,17.2,3.9,15.0,7.6,10.3,15.2,17.1,8.1,32.1
2,pecuario,livestock,9.5,5.6,10.6,0.9,-1.3,2.4,4.0,4.0,4.5,3.8,-0.1,-3.1,-2.2,-5.1,0.0,0.6,1.3
3,pesca,fishing,-27.0,-23.6,-14.6,-30.9,30.7,-32.7,2.0,15.3,17.8,17.5,-7.6,-11.1,-52.3,49.0,77.6,5.2,-17.3
4,mineria,mining and fuel,4.4,1.7,3.1,-0.3,3.3,1.7,1.8,2.5,4.0,7.3,2.9,2.0,7.0,1.9,3.0,3.1,0.7
5,mineria metalica,metals,10.4,3.9,8.3,3.3,6.7,5.4,6.2,8.3,8.7,12.2,6.4,-0.1,7.0,2.3,2.5,5.7,0.1
6,petroleo crudo,fuels,-5.0,-2.0,-4.4,-5.8,-2.3,-4.2,-4.9,-5.7,-3.4,-1.0,-3.0,5.5,6.9,1.3,3.9,-1.2,1.8
7,manufactura,manufacturing,-3.0,-3.9,4.5,-3.7,9.0,-1.6,0.3,2.9,2.2,7.4,-0.2,0.3,-3.9,8.1,11.8,2.6,2.4
8,procesadores recursos primarios,based on raw materials,-10.7,-9.0,-6.1,-19.3,16.5,-6.6,-0.2,5.9,9.7,13.0,4.8,7.1,-12.5,22.5,32.6,5.3,-1.9
9,resto industria,non primary,0.8,-1.7,9.5,5.6,6.5,-0.1,0.6,1.5,-0.9,5.5,-1.8,-1.8,-0.4,1.8,3.2,1.6,4.3


In [13]:
old_dataframes_dict_1['ns_13_1997_1'].head(30)

,year,id_ns,date,sectores_economicos,economic_sectors,1995_nov,1995_dic,1995_year,1996_ene,1996_feb,...,1996_may,1996_jun,1996_jul,1996_ago,1996_sep,1996_oct,1996_nov,1996_dic,1996_ene_dic,1997_ene
0,1997,13,1997-04-11,agropecuario,agriculture and livestock,-0.1,3.5,7.6,-3.6,0.3,...,2.8,14.4,3.8,9.8,3.3,4.7,5.4,9.3,5.6,17.1
1,1997,13,1997-04-11,agricola,agriculture,-7.8,1.7,6.1,-7.4,1.6,...,2.5,17.2,3.9,15.0,7.6,10.3,15.2,17.1,8.1,32.1
2,1997,13,1997-04-11,pecuario,livestock,9.5,5.6,10.6,0.9,-1.3,...,4.0,4.5,3.8,-0.1,-3.1,-2.2,-5.1,0.0,0.6,1.3
3,1997,13,1997-04-11,pesca,fishing,-27.0,-23.6,-14.6,-30.9,30.7,...,15.3,17.8,17.5,-7.6,-11.1,-52.3,49.0,77.6,5.2,-17.3
4,1997,13,1997-04-11,mineria e hidrocarburos,mining and fuel,4.4,1.7,3.1,-0.3,3.3,...,2.5,4.0,7.3,2.9,2.0,7.0,1.9,3.0,3.1,0.7
5,1997,13,1997-04-11,mineria metalica,metals,10.4,3.9,8.3,3.3,6.7,...,8.3,8.7,12.2,6.4,-0.1,7.0,2.3,2.5,5.7,0.1
6,1997,13,1997-04-11,petroleo crudo,fuels,-5.0,-2.0,-4.4,-5.8,-2.3,...,-5.7,-3.4,-1.0,-3.0,5.5,6.9,1.3,3.9,-1.2,1.8
7,1997,13,1997-04-11,manufactura,manufacturing,-3.0,-3.9,4.5,-3.7,9.0,...,2.9,2.2,7.4,-0.2,0.3,-3.9,8.1,11.8,2.6,2.4
8,1997,13,1997-04-11,procesadores recursos primarios,based on raw materials,-10.7,-9.0,-6.1,-19.3,16.5,...,5.9,9.7,13.0,4.8,7.1,-12.5,22.5,32.6,5.3,-1.9
9,1997,13,1997-04-11,resto industria,non primary,0.8,-1.7,9.5,5.6,6.5,...,1.5,-0.9,5.5,-1.8,-1.8,-0.4,1.8,3.2,1.6,4.3


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    <span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">Table 2.</span> Extraction and cleaning of data from tables on quarterly and annual real GDP growth rates.
    </span>
    </h3>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Please check that the flat file <b>"ns_dates.csv"</b> is updated with the dates, years and ids for the newly downloaded PDF <span style="font-size: 24px;">&#128462;</span> (WR). That file is located in the <b>"ns_dates"</b> folder and is uploaded to SQL from the jupyeter notebook <code>aux_files_to_sql.ipynb</code>
    </span>
</div>

In [14]:
# Set the locale to Spanish
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Dictionary to store generated DataFrames
old_dataframes_dict_2 = {}

# Path for the processed folders log file
record_path = 'dataframes_record/old_processed_folders_2.txt'

# Function to correct month names
def correct_month_name(month):
    months_mapping = {
        'setiembre': 'septiembre',
        # Add more mappings as needed for other month names
    }
    return months_mapping.get(month, month)

# Function to register processed folder
def register_processed_folder(folder, num_processed_files):
    with open(record_path, 'a') as file:
        file.write(f"{folder}:{num_processed_files}\n")

# Function to check if folder has been processed
def folder_processed(folder):
    if not os.path.exists(record_path):
        return False
    with open(record_path, 'r') as file:
        for line in file:
            if line.startswith(folder):
                return True
    return False

# Function to fetch date from database
def get_date(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    date_result = pd.read_sql(query, engine)
    return date_result.iloc[0, 0] if not date_result.empty else None

def process_csv(csv_path, engine):
    old_tables_dict_2 = {}  # Local dictionary for each CSV
    table_counter = 2

    filename = os.path.basename(csv_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No matches found for id_ns and year in filename:", filename)
        return None, None, None, None # Return None for tables_dict_1 as well

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    dataframe_name = f"{new_filename}_{table_counter}"
    old_tables_dict_2[dataframe_name] = df.copy()

    # Apply cleanup functions to a copy of the DataFrame
    df_clean = df.copy()

    # Conditional cleaning based on the first column
    if df_clean.columns[1] == 'economic_sectors':
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)
    else:
        # Cleaning functions (set as required)
        df_clean = replace_total_with_year(df_clean)
        df_clean = drop_nan_rows(df_clean)
        df_clean = drop_nan_columns(df_clean)
        year_columns = extract_years(df_clean)
        df_clean = roman_arabic(df_clean)
        df_clean = fix_duplicates(df_clean)
        df_clean = relocate_last_column(df_clean)
        df_clean = replace_first_row_nan(df_clean)
        df_clean = clean_first_row(df_clean)
        df_clean = get_quarters_sublist_list(df_clean, year_columns)
        df_clean = reset_index(df_clean)
        df_clean = first_row_columns(df_clean)
        df_clean = clean_columns_values(df_clean)
        df_clean = reset_index(df_clean)
        df_clean = convert_float(df_clean)
        df_clean = replace_set_sep(df_clean)
        df_clean = spaces_se_es(df_clean)
        df_clean = replace_mineria(df_clean)
        df_clean = replace_mining(df_clean)
        df_clean = rounding_values(df_clean, decimals=1)

    # Add the column 'year' to the clean DataFrame
    df_clean.insert(0, 'year', year)
    
    # Add the column 'id_ns' to the clean DataFrame
    df_clean.insert(1, 'id_ns', id_ns)
    
    # Get corresponding date from database
    date = get_date(df_clean, engine)
    if date:
        # Add 'date' column to cleaned DataFrame
        df_clean.insert(2, 'date', date)
    else:
        print("Date not found in database for id_ns:", id_ns, "and year:", year)

    # Store cleaned DataFrame in old_dataframes_dict_2
    old_dataframes_dict_2[dataframe_name] = df_clean

    return id_ns, year, old_tables_dict_2

# Function to process folder
def process_folder(folder, engine):
    print(f"Processing folder {os.path.basename(folder)}")
    csv_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]

    num_csv_processed = 0
    num_dataframes_generated = 0

    table_counter = 1  # Initialize table counter here
    old_tables_dict_2 = {}  # Declare old_tables_dict_1 outside main loop
    
    for csv_file in csv_files:
        id_ns, year, tables_dict_temp = process_csv(csv_file, engine)

        if tables_dict_temp:
            for dataframe_name, df in tables_dict_temp.items():
                file_name = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                dataframe_name = f"{file_name}_2"
                
                # Store raw DataFrame in old_tables_dict_2
                old_tables_dict_2[dataframe_name] = df.copy()
                
                # Procesar y limpiar el DataFrame
                df_clean = df.copy()
                
                # Conditional cleaning based on the first column
                if df_clean.columns[1] == 'economic_sectors':
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                else:
                    # Cleaning functions (set as required)
                    df_clean = replace_total_with_year(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = roman_arabic(df_clean)
                    df_clean = fix_duplicates(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = replace_first_row_nan(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = get_quarters_sublist_list(df_clean, year_columns)
                    df_clean = reset_index(df_clean)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                
                # Add the column 'year' to the clean DataFrame
                df_clean.insert(0, 'year', year)

                # Add the column 'id_ns' to the clean DataFrame
                df_clean.insert(1, 'id_ns', id_ns)

                # Get corresponding date from database
                date = get_date(df_clean, engine)
                if date:
                    # Add 'date' column to cleaned DataFrame
                    df_clean.insert(2, 'date', date)
                else:
                    print("Date not found in database for id_ns:", id_ns, "and year:", year)

                # Store cleaned DataFrame in old_dataframes_dict_2
                old_dataframes_dict_2[dataframe_name] = df_clean

                print(f'  {table_counter}. The dataframe generated for the {csv_file} file is: {dataframe_name}')
                num_dataframes_generated += 1
                table_counter += 1  # Increment table counter here
        
        num_csv_processed += 1  # Increment number of processed CSV for each CSV in folder

    return num_csv_processed, num_dataframes_generated, old_tables_dict_2

def process_folders():
    csv_folder = table_2_folder
    folders = [os.path.join(csv_folder, d) for d in os.listdir(csv_folder) if os.path.isdir(os.path.join(csv_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_2 = {}  # Initialize old_tables_dict_1 here
    
    for folder in folders:
        if folder_processed(folder):
            print(f"Folder {folder} has already been processed.")
            continue
        
        num_csv_processed, num_dataframes_generated, tables_dict_temp = process_folder(folder, engine)
        
        # Update old_tables_dict_2 with values returned from process_folder()
        old_tables_dict_2.update(tables_dict_temp)
        
        register_processed_folder(folder, num_csv_processed)
    
        # Ask user if they want to continue with next folder
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Ensure the messagebox is in front
        message = f"Process {folder} complete. Processed {num_csv_processed} CSV(s) and generated {num_dataframes_generated} DataFrame(s). Continue with next folder?"
        if not messagebox.askyesno("Continue?", message):
            break

    print("Processing completed for all folders.")  # Add a message to indicate completion

    return old_tables_dict_2
    
if __name__ == "__main__":
    engine = create_sqlalchemy_engine()
    old_tables_dict_2 = process_folders()

Processing folder 1997
  1. The dataframe generated for the input_data\table_2\1997\ns-01-1997.csv file is: ns_01_1997_2
  2. The dataframe generated for the input_data\table_2\1997\ns-02-1997.csv file is: ns_02_1997_2
  3. The dataframe generated for the input_data\table_2\1997\ns-03-1997.csv file is: ns_03_1997_2
  4. The dataframe generated for the input_data\table_2\1997\ns-04-1997.csv file is: ns_04_1997_2
  5. The dataframe generated for the input_data\table_2\1997\ns-05-1997.csv file is: ns_05_1997_2
  6. The dataframe generated for the input_data\table_2\1997\ns-06-1997.csv file is: ns_06_1997_2
  7. The dataframe generated for the input_data\table_2\1997\ns-07-1997.csv file is: ns_07_1997_2
  8. The dataframe generated for the input_data\table_2\1997\ns-08-1997.csv file is: ns_08_1997_2
  9. The dataframe generated for the input_data\table_2\1997\ns-09-1997.csv file is: ns_09_1997_2
  10. The dataframe generated for the input_data\table_2\1997\ns-10-1997.csv file is: ns_10_199

  37. The dataframe generated for the input_data\table_2\1998\ns-37-1998.csv file is: ns_37_1998_2
  38. The dataframe generated for the input_data\table_2\1998\ns-38-1998.csv file is: ns_38_1998_2
  39. The dataframe generated for the input_data\table_2\1998\ns-39-1998.csv file is: ns_39_1998_2
  40. The dataframe generated for the input_data\table_2\1998\ns-40-1998.csv file is: ns_40_1998_2
  41. The dataframe generated for the input_data\table_2\1998\ns-41-1998.csv file is: ns_41_1998_2
  42. The dataframe generated for the input_data\table_2\1998\ns-42-1998.csv file is: ns_42_1998_2
  43. The dataframe generated for the input_data\table_2\1998\ns-43-1998.csv file is: ns_43_1998_2
  44. The dataframe generated for the input_data\table_2\1998\ns-44-1998.csv file is: ns_44_1998_2
  45. The dataframe generated for the input_data\table_2\1998\ns-45-1998.csv file is: ns_45_1998_2
  46. The dataframe generated for the input_data\table_2\1998\ns-46-1998.csv file is: ns_46_1998_2
  47. The 

  23. The dataframe generated for the input_data\table_2\2000\ns-23-2000.csv file is: ns_23_2000_2
  24. The dataframe generated for the input_data\table_2\2000\ns-24-2000.csv file is: ns_24_2000_2
  25. The dataframe generated for the input_data\table_2\2000\ns-25-2000.csv file is: ns_25_2000_2
  26. The dataframe generated for the input_data\table_2\2000\ns-26-2000.csv file is: ns_26_2000_2
  27. The dataframe generated for the input_data\table_2\2000\ns-27-2000.csv file is: ns_27_2000_2
  28. The dataframe generated for the input_data\table_2\2000\ns-28-2000.csv file is: ns_28_2000_2
  29. The dataframe generated for the input_data\table_2\2000\ns-29-2000.csv file is: ns_29_2000_2
  30. The dataframe generated for the input_data\table_2\2000\ns-30-2000.csv file is: ns_30_2000_2
  31. The dataframe generated for the input_data\table_2\2000\ns-31-2000.csv file is: ns_31_2000_2
  32. The dataframe generated for the input_data\table_2\2000\ns-32-2000.csv file is: ns_32_2000_2
  33. The 

  14. The dataframe generated for the input_data\table_2\2002\ns-14-2002.csv file is: ns_14_2002_2
  15. The dataframe generated for the input_data\table_2\2002\ns-15-2002.csv file is: ns_15_2002_2
  16. The dataframe generated for the input_data\table_2\2002\ns-16-2002.csv file is: ns_16_2002_2
  17. The dataframe generated for the input_data\table_2\2002\ns-17-2002.csv file is: ns_17_2002_2
  18. The dataframe generated for the input_data\table_2\2002\ns-18-2002.csv file is: ns_18_2002_2
  19. The dataframe generated for the input_data\table_2\2002\ns-19-2002.csv file is: ns_19_2002_2
  20. The dataframe generated for the input_data\table_2\2002\ns-20-2002.csv file is: ns_20_2002_2
  21. The dataframe generated for the input_data\table_2\2002\ns-21-2002.csv file is: ns_21_2002_2
  22. The dataframe generated for the input_data\table_2\2002\ns-22-2002.csv file is: ns_22_2002_2
  23. The dataframe generated for the input_data\table_2\2002\ns-23-2002.csv file is: ns_23_2002_2
  24. The 

Processing folder 2004
  1. The dataframe generated for the input_data\table_2\2004\ns-01-2004.csv file is: ns_01_2004_2
  2. The dataframe generated for the input_data\table_2\2004\ns-02-2004.csv file is: ns_02_2004_2
  3. The dataframe generated for the input_data\table_2\2004\ns-03-2004.csv file is: ns_03_2004_2
  4. The dataframe generated for the input_data\table_2\2004\ns-04-2004.csv file is: ns_04_2004_2
  5. The dataframe generated for the input_data\table_2\2004\ns-05-2004.csv file is: ns_05_2004_2
  6. The dataframe generated for the input_data\table_2\2004\ns-06-2004.csv file is: ns_06_2004_2
  7. The dataframe generated for the input_data\table_2\2004\ns-07-2004.csv file is: ns_07_2004_2
  8. The dataframe generated for the input_data\table_2\2004\ns-08-2004.csv file is: ns_08_2004_2
  9. The dataframe generated for the input_data\table_2\2004\ns-09-2004.csv file is: ns_09_2004_2
  10. The dataframe generated for the input_data\table_2\2004\ns-10-2004.csv file is: ns_10_200

  38. The dataframe generated for the input_data\table_2\2005\ns-38-2005.csv file is: ns_38_2005_2
  39. The dataframe generated for the input_data\table_2\2005\ns-39-2005.csv file is: ns_39_2005_2
  40. The dataframe generated for the input_data\table_2\2005\ns-40-2005.csv file is: ns_40_2005_2
  41. The dataframe generated for the input_data\table_2\2005\ns-41-2005.csv file is: ns_41_2005_2
  42. The dataframe generated for the input_data\table_2\2005\ns-42-2005.csv file is: ns_42_2005_2
  43. The dataframe generated for the input_data\table_2\2005\ns-43-2005.csv file is: ns_43_2005_2
  44. The dataframe generated for the input_data\table_2\2005\ns-44-2005.csv file is: ns_44_2005_2
  45. The dataframe generated for the input_data\table_2\2005\ns-45-2005.csv file is: ns_45_2005_2
  46. The dataframe generated for the input_data\table_2\2005\ns-46-2005.csv file is: ns_46_2005_2
  47. The dataframe generated for the input_data\table_2\2005\ns-47-2005.csv file is: ns_47_2005_2
  48. The 

  30. The dataframe generated for the input_data\table_2\2007\ns-30-2007.csv file is: ns_30_2007_2
  31. The dataframe generated for the input_data\table_2\2007\ns-31-2007.csv file is: ns_31_2007_2
  32. The dataframe generated for the input_data\table_2\2007\ns-32-2007.csv file is: ns_32_2007_2
  33. The dataframe generated for the input_data\table_2\2007\ns-33-2007.csv file is: ns_33_2007_2
  34. The dataframe generated for the input_data\table_2\2007\ns-34-2007.csv file is: ns_34_2007_2
  35. The dataframe generated for the input_data\table_2\2007\ns-35-2007.csv file is: ns_35_2007_2
  36. The dataframe generated for the input_data\table_2\2007\ns-36-2007.csv file is: ns_36_2007_2
  37. The dataframe generated for the input_data\table_2\2007\ns-37-2007.csv file is: ns_37_2007_2
  38. The dataframe generated for the input_data\table_2\2007\ns-38-2007.csv file is: ns_38_2007_2
  39. The dataframe generated for the input_data\table_2\2007\ns-39-2007.csv file is: ns_39_2007_2
  40. The 

  16. The dataframe generated for the input_data\table_2\2009\ns-16-2009.csv file is: ns_16_2009_2
  17. The dataframe generated for the input_data\table_2\2009\ns-17-2009.csv file is: ns_17_2009_2
  18. The dataframe generated for the input_data\table_2\2009\ns-18-2009.csv file is: ns_18_2009_2
  19. The dataframe generated for the input_data\table_2\2009\ns-19-2009.csv file is: ns_19_2009_2
  20. The dataframe generated for the input_data\table_2\2009\ns-20-2009.csv file is: ns_20_2009_2
  21. The dataframe generated for the input_data\table_2\2009\ns-21-2009.csv file is: ns_21_2009_2
  22. The dataframe generated for the input_data\table_2\2009\ns-22-2009.csv file is: ns_22_2009_2
  23. The dataframe generated for the input_data\table_2\2009\ns-23-2009.csv file is: ns_23_2009_2
  24. The dataframe generated for the input_data\table_2\2009\ns-24-2009.csv file is: ns_24_2009_2
  25. The dataframe generated for the input_data\table_2\2009\ns-25-2009.csv file is: ns_25_2009_2
  26. The 

Processing folder 2011
  1. The dataframe generated for the input_data\table_2\2011\ns-01-2011.csv file is: ns_01_2011_2
  2. The dataframe generated for the input_data\table_2\2011\ns-02-2011.csv file is: ns_02_2011_2
  3. The dataframe generated for the input_data\table_2\2011\ns-03-2011.csv file is: ns_03_2011_2
  4. The dataframe generated for the input_data\table_2\2011\ns-04-2011.csv file is: ns_04_2011_2
  5. The dataframe generated for the input_data\table_2\2011\ns-05-2011.csv file is: ns_05_2011_2
  6. The dataframe generated for the input_data\table_2\2011\ns-06-2011.csv file is: ns_06_2011_2
  7. The dataframe generated for the input_data\table_2\2011\ns-07-2011.csv file is: ns_07_2011_2
  8. The dataframe generated for the input_data\table_2\2011\ns-08-2011.csv file is: ns_08_2011_2
  9. The dataframe generated for the input_data\table_2\2011\ns-09-2011.csv file is: ns_09_2011_2
  10. The dataframe generated for the input_data\table_2\2011\ns-10-2011.csv file is: ns_10_201

  37. The dataframe generated for the input_data\table_2\2012\ns-37-2012.csv file is: ns_37_2012_2
  38. The dataframe generated for the input_data\table_2\2012\ns-38-2012.csv file is: ns_38_2012_2
  39. The dataframe generated for the input_data\table_2\2012\ns-39-2012.csv file is: ns_39_2012_2
  40. The dataframe generated for the input_data\table_2\2012\ns-40-2012.csv file is: ns_40_2012_2
  41. The dataframe generated for the input_data\table_2\2012\ns-41-2012.csv file is: ns_41_2012_2
  42. The dataframe generated for the input_data\table_2\2012\ns-42-2012.csv file is: ns_42_2012_2
  43. The dataframe generated for the input_data\table_2\2012\ns-43-2012.csv file is: ns_43_2012_2
  44. The dataframe generated for the input_data\table_2\2012\ns-44-2012.csv file is: ns_44_2012_2
  45. The dataframe generated for the input_data\table_2\2012\ns-45-2012.csv file is: ns_45_2012_2
  46. The dataframe generated for the input_data\table_2\2012\ns-46-2012.csv file is: ns_46_2012_2
  47. The 

In [15]:
old_tables_dict_2['ns_44_2004_2']

,sectores_economicos,economic_sectors,2002_1,2002_2,2002_3,2002_4,2002_year,2003_1,2003_2,2003_3,2003_4,2003_year,2004_1,2004_2,2004_3
0,agropecuario,agriculture and livestock,6.4,7.2,3.8,6.3,6.0,2.2,4.0,2.5,-0.4,2.3,1.7,-4.7,-1.0
1,agrícola,agriculture,8.6,6.8,1.8,6.9,6.1,0.6,3.8,1.7,-2.0,1.6,1.2,-7.3,-3.4
2,pecuario,livestock,2.5,7.2,6.4,5.2,5.3,4.0,3.8,3.1,1.3,3.0,2.3,0.9,1.8
3,pesca,fishing,-18.4,9.1,8.4,29.5,5.7,-12.8,-13.8,-17.3,-7.5,-12.6,13.1,26.0,33.7
4,minería e hidrocarburos,mining and fuel,27.1,20.5,5.0,2.7,12.5,4.5,10.3,7.9,4.9,6.9,12.5,2.6,1.5
5,minería metálica,metals,28.3,20.8,4.3,1.8,12.3,5.0,11.1,9.2,5.6,7.7,13.9,3.1,0.6
6,hidrocarburos,fuels,5.2,0.5,-0.8,-2.2,0.6,-6.2,-1.2,-7.3,-3.1,-4.5,-5.4,-4.3,12.7
7,manufactura,manufacturing,0.5,4.7,4.8,5.9,4.0,5.6,0.8,1.8,0.6,2.1,4.6,6.1,6.1
8,procesadores de recursos primarios,based on raw materials,-6.2,-0.5,-2.6,9.0,0.0,-0.7,-2.3,0.8,-4.4,-1.8,1.4,6.8,1.5
9,industria no primaria,non primary industries,2.4,6.4,6.9,5.1,5.2,7.2,1.7,2.1,2.1,3.2,5.3,5.9,7.1


In [16]:
old_dataframes_dict_2['ns_44_2004_2']

,year,id_ns,date,sectores_economicos,economic_sectors,2002_1,2002_2,2002_3,2002_4,2002_year,2003_1,2003_2,2003_3,2003_4,2003_year,2004_1,2004_2,2004_3
0,2004,44,2004-11-19,agropecuario,agriculture and livestock,6.4,7.2,3.8,6.3,6.0,2.2,4.0,2.5,-0.4,2.3,1.7,-4.7,-1.0
1,2004,44,2004-11-19,agricola,agriculture,8.6,6.8,1.8,6.9,6.1,0.6,3.8,1.7,-2.0,1.6,1.2,-7.3,-3.4
2,2004,44,2004-11-19,pecuario,livestock,2.5,7.2,6.4,5.2,5.3,4.0,3.8,3.1,1.3,3.0,2.3,0.9,1.8
3,2004,44,2004-11-19,pesca,fishing,-18.4,9.1,8.4,29.5,5.7,-12.8,-13.8,-17.3,-7.5,-12.6,13.1,26.0,33.7
4,2004,44,2004-11-19,mineria e hidrocarburos,mining and fuel,27.1,20.5,5.0,2.7,12.5,4.5,10.3,7.9,4.9,6.9,12.5,2.6,1.5
5,2004,44,2004-11-19,mineria metalica,metals,28.3,20.8,4.3,1.8,12.3,5.0,11.1,9.2,5.6,7.7,13.9,3.1,0.6
6,2004,44,2004-11-19,hidrocarburos,fuels,5.2,0.5,-0.8,-2.2,0.6,-6.2,-1.2,-7.3,-3.1,-4.5,-5.4,-4.3,12.7
7,2004,44,2004-11-19,manufactura,manufacturing,0.5,4.7,4.8,5.9,4.0,5.6,0.8,1.8,0.6,2.1,4.6,6.1,6.1
8,2004,44,2004-11-19,procesadores de recursos primarios,based on raw materials,-6.2,-0.5,-2.6,9.0,0.0,-0.7,-2.3,0.8,-4.4,-1.8,1.4,6.8,1.5
9,2004,44,2004-11-19,industria no primaria,non primary industries,2.4,6.4,6.9,5.1,5.2,7.2,1.7,2.1,2.1,3.2,5.3,5.9,7.1


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Real-time data of Peru's GDP growth rates</span></h1>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
This section creates the GDP growth rate vintages for Peru using <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 1</a> and <a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 2</a>, which were extracted and cleaned in the previous section. Each table from each WR (CSV <span style="font-size: 24px;">&#128452;</span>) was extracted and cleaned individually in the previous section. Here, we will concatenate all the tables for a specific economic sector, thus creating a vintage dataset of (real) GDP growth by economic sector from <b>1994</b> to <b>2012</b> (<a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 1</a>) and <b>1997</b> to <b>2012</b> (<a href="#3-2-1" style="color: rgb(0, 153, 123); font-size: 16px;">Table 2</a>).
<div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
As preferred or as appropriate, you can create the data manually, step by step, or focus on specific sectors or frequencies. Alternatively, you can choose a more efficient or automated approach by generating the data for all sectors and frequencies simultaneously.
<div/>

<div id="3-1">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Manual process of data creation in real time: sector by sector and frequency by frequency.
    </span>
    </h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    With this method you can create and inspect the dataset sector by sector and frequency by frequency. This is useful if you want to create data only for particular sectors and frequencies.
<div/>

<div id="select_sector">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">$\bullet$</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Select <code>sector_economico</code> and <code>economic_sector</code></span></h1>
    </div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">
<p>     
When executing the following code, a window will be displayed with options in <b>Spanish</b> and <b>English</b> to select <b>economic sectors</b>. Choose them to concatenate Peru GDP growth rates (annual, quarterly or monthly) by sector.
</p>
<div/>

In [10]:
# Call the function to display the window and capture the selected values
selected_spanish, selected_english, sector = show_option_window()

# Display the selected values
print(f"You have selected sector = {sector}, selected_spanish = {selected_spanish}, and selected_english = {selected_english}.")

You have selected sector = gdp, selected_spanish = pbi, and selected_english = gdp.


<div id="select_freq">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">$\bullet$</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Select <code>frequency</code></span></h1>
    </div>

In [ ]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

<div id="counter">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">$\bullet$</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Set counter (dataframe name suffix)</span></h1>
    </div>

In [ ]:
# Call the function to set the counter
if frequency == "monthly":
    counter = 1
elif frequency == "quarterly":
    counter = 2
elif frequency == "annual":
    counter = 2
else:
    counter = None 

print(counter)

<div id="3-1-1">
   <!-- Contenido de la celda de destino -->
</div>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Growth rates datasets concatenation for all frequencies
    </span>
    </h3>

In [ ]:
# Dynamically construct the function name and dictionary name
function_name = f"concatenate_{frequency}_df"
dataframe_dict_name = f"old_dataframes_dict_{counter}"

# Check that both the function and dictionary exist in the global scope
if function_name in globals() and dataframe_dict_name in globals():
    # Call the function using its reference from globals()
    globals()[f"old_{sector}_{frequency}_growth_rates"] = globals()[function_name](
        globals()[dataframe_dict_name], selected_spanish, selected_english
    )
else:
    print(f"Error: {function_name} or {dataframe_dict_name} does not exist in the global scope.")

In [ ]:
#pd.set_option('display.max_rows', None)
globals()[f"old_{sector}_{frequency}_growth_rates"].head(10)

<div id="3-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<h3><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Uploading data to SQL</span></h3> 

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
Finally, we upload all the datasets generated in this jupyter notebook to the <code>'gdp_revisions_datasets'</code> database of <code>PostgresSQL</code>.
<div/>

In [ ]:
engine = create_sqlalchemy_engine()

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
Loading
<div/>

In [ ]:
globals()[f"old_{sector}_{frequency}_growth_rates"].to_sql(f'old_{sector}_{frequency}_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#select_sector" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#select_sector" style="color: rgb(255, 32, 78); text-decoration: none;">Back to select sectors.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#select_freq" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#select_freq" style="color: rgb(255, 32, 78); text-decoration: none;">Back to select frequency.</a>
</div>

<div id="3-2">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Automatic data creation process in real time: all sectors and frequencies at the same time.
    </span>
    </h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    With this method you can create the dataset for all sectors and all frequencies at the same time. This is more efficient if the goal is to generate all possible combinations of datasets for <code>sector</code> and <code>frequency</code> (without excluding any sector or frequency).
<div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    List of frequencies to be used to create concatenated datasets
    <div/>

In [17]:
frequencies = [
        "monthly", 
        "quarterly",
        "annual"
    ]

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Function to process growth rates datasets: concatenate and load to SQL
    <div/>

In [18]:
def process_old_datasets_to_sql(sector, frequency):
    # Initialize counter for loaded DataFrames
    total_loaded = 0

    # Set counter based on frequency
    if frequency == "monthly":
        counter = 1
    elif frequency in ["quarterly", "annual"]:
        counter = 2
    else:
        print(f"Unknown frequency: {frequency}")
        return None

    # Dynamically build function and dictionary names
    function_name = f"concatenate_{frequency}_df"
    dataframe_dict_name = f"old_dataframes_dict_{counter}"

    if function_name in globals() and dataframe_dict_name in globals():
        # Generate the DataFrame
        df_name = f"old_{sector}_{frequency}_growth_rates"
        globals()[df_name] = globals()[function_name](
            globals()[dataframe_dict_name], option_mapping[sector][0], option_mapping[sector][1]
        )

        # Load to SQL
        engine = create_sqlalchemy_engine()
        globals()[df_name].to_sql(df_name, engine, index=False, if_exists='replace')

        return globals()[df_name]
    else:
        print(f"Error: {function_name} or {dataframe_dict_name} does not exist in the global scope.")
        return None

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Run the function to create concatenated datasets for all sectors and frequencies and load to SQL
    <div/>

In [19]:
# Initialize counter
processed_datasets = 0

# Process all combinations
for sector in option_mapping.keys():
    for frequency in frequencies:
        print(f"Processing {sector} - {frequency}")
        df = process_old_datasets_to_sql(sector, frequency)
        if df is not None:
            display(df.head(10))  # Display the first 10 rows
            processed_datasets += 1  # Increment counter

# Display total number of processed datasets
print(f"Total datasets processed: {processed_datasets}")

Processing gdp - monthly
Number of rows in the concatenated dataframe: 939


,year,id_ns,date,ene_1992,feb_1992,mar_1992,abr_1992,may_1992,jun_1992,jul_1992,...,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012
0,1994,01,1994-01-10,1.3,1.8,2.8,-7.5,-5.7,-3.1,-7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1994,02,1994-01-17,1.3,1.8,2.8,-7.5,-5.7,-3.1,-7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1994,03,1994-01-24,1.3,1.8,2.8,-7.5,-5.7,-3.1,-7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1994,04,1994-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1994,05,1994-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1994,06,1994-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1994,07,1994-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1994,08,1994-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1994,09,1994-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1994,10,1994-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Processing gdp - quarterly
Number of rows in the concatenated dataframe: 787


,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,...,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3
0,1997,01,1997-01-10,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,02,1997-01-17,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,03,1997-01-24,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,04,1997-01-31,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,05,1997-02-07,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,06,1997-02-14,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,07,1997-02-21,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,08,1997-02-28,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1997,09,1997-03-07,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1997,10,1997-03-14,10.7,16.1,11.0,14.4,12.7,8.7,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Processing gdp - annual
Number of rows in the concatenated dataframe: 787


...

Processing services - quarterly
Number of rows in the concatenated dataframe: 764


,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,...,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3
0,1997,24,1997-06-27,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,25,1997-07-04,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,26,1997-07-11,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,27,1997-07-18,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,28,1997-07-25,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,29,1997-08-08,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,30,1997-08-15,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,31,1997-08-22,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1997,32,1997-08-29,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1997,33,1997-09-05,NaN,NaN,NaN,NaN,10.9,8.1,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Processing services - annual
Number of rows in the concatenated dataframe: 764


,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,...,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011
0,1997,24,1997-06-27,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,25,1997-07-04,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,26,1997-07-11,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,27,1997-07-18,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,28,1997-07-25,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,29,1997-08-08,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,30,1997-08-15,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,31,1997-08-22,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1997,32,1997-08-29,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1997,33,1997-09-05,NaN,NaN,7.0,2.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Total datasets processed: 27


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div style="font-size: 16px; background-color: #F5F5F5; padding: 18px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Once you have all the datasets generated by this script (<code>old_gdp_datasets.ipynb</code>) you can concatenate with those generated in the script <code>new_gdp_datasets.ipynb</code>. <b>Section 6</b> of the script <code>aux_files_to_sql.ipynb</code> concatenates both <b>new</b> and <b>old</b> datasets for <b>all sectors</b> and <b>all frequencies</b>.
</div>

---
---
